In [1]:
#imports for the <i class="fas fa-project-diagram    "></i>
#1
import pandas as pd

#2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


Cleaning the large data set to remove all players that are below 15 minutes per game and make all stats per game. I made all stats per game stats. I didn't put teams, I will add conferences in the future, trying to get the 30 best players for now, might alter this project to ALL NBA TEAMS, and awards instead, as the dataset is better for that, as its a full year dataset.

In [ ]:
#getting dataset
filepath = "2023_nba_player_stats.csv"
df_original = pd.read_csv(filepath)

# Step 1: Compute Minutes Per Game (MPG)
df_original["MPG"] = df_original["Min"] / df_original["GP"]

# Step 2: Filter out players who averaged less than 15 minutes per game
df_filtered = df_original[df_original["MPG"] >= 15]

# Step 3: Convert key stats to per-game values (excluding "+/-")
per_game_stats = ["PTS", "AST", "REB", "TOV", "STL", "BLK"]
for stat in per_game_stats:
    df_filtered[stat] = df_filtered[stat] / df_filtered["GP"]

# Step 4: Drop "Min" and select final columns
final_columns = [
    "PName",  # Player name
    "POS",    # Position
    "GP",     # Games Played
    "MPG",    # Minutes per game
    "PTS",    # Points per game
    "AST",    # Assists per game
    "REB",    # Rebounds per game
    "FG%",    # Field goal %
    "3P%",    # Three-point %
    "TOV",    # Turnovers per game
    "STL",    # Steals per game
    "BLK",    # Blocks per game
    "+/-"     # Plus-minus (not per game)
]

df_final = df_filtered[final_columns]

# Save the final cleaned and transformed dataset
output_path = "nba_cleaned_full_pipeline.csv"  # Make sure the 'data' folder exists
df_final.to_csv(output_path, index=False)



C:\Users\victo\AppData\Local\Temp\ipykernel_29472\823274415.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[stat] = df_filtered[stat] / df_filtered["GP"]


Now gonna preprocess the data

In [ ]:
#2
# Load the dataset
df = pd.read_csv("nba_cleaned_full_pipeline.csv")

# Create 'is_all_star' column using official 2023 NBA All-Stars
all_stars_2023 = [
    "LeBron James", "Giannis Antetokounmpo", "Stephen Curry", "Luka Doncic", "Nikola Jokic",
    "Jayson Tatum", "Joel Embiid", "Ja Morant", "Donovan Mitchell", "Lauri Markkanen",
    "Shai Gilgeous-Alexander", "Jaren Jackson Jr.", "Domantas Sabonis", "Tyrese Haliburton",
    "De'Aaron Fox", "Bam Adebayo", "Jrue Holiday", "DeMar DeRozan", "Jaylen Brown",
    "Paul George", "Pascal Siakam", "Anthony Edwards"
]
df["is_all_star"] = df["PName"].apply(lambda x: 1 if x in all_stars_2023 else 0) #edit this to make more readable

# Drop non-feature columns
X = df.drop(columns=["PName", "POS", "is_all_star"])
y = df["is_all_star"]

# Train/Validation/Test split (70/15/15), stratified by label
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Print basic info
print("Train size:", len(X_train))
print("Validation size:", len(X_val))
print("Test size:", len(X_test))
print("Train all-star ratio:", round(y_train.mean(), 3))
print("Validation all-star ratio:", round(y_val.mean(), 3))
print("Test all-star ratio:", round(y_test.mean(), 3))


Train size: 239
Validation size: 51
Test size: 52
Train all-star ratio: 0.063
Validation all-star ratio: 0.059
Test all-star ratio: 0.077
